In [32]:
import xml.etree.cElementTree as ET
import pprint
import re
import collections
import codecs
import cerberus
import schema
import csv

In [33]:
OSM_FILE = "Seattlemap"  


In [34]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

# Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'w') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

# Tag Types

In [35]:
#Patters to search for tag attribute 'K' with lower alphabets, colons and problem characters
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
    

In [36]:
#Counting the lower, lower_colon and problem chars
def key_type(element, keys):
    if element.tag == "tag":
        # YOUR CODE HERE
        l = lower.search(element.attrib['k'])
        lc = lower_colon.search(element.attrib['k'])
        pc = problemchars.search(element.attrib['k'])
        if l:
            keys["lower"] += 1
        elif lc:
            keys["lower_colon"] += 1
        elif pc:
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
        
        
    return keys





In [37]:
#defining keys dictionary
def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


In [38]:
#testing the count of lower , lower_colon and problem chars
def test():
    # You can use another testfile 'map.osm' to look at your solution
    # Note that the assertion below will be incorrect then.
    # Note as well that the test function here is only used in the Test Run;
    # when you submit, your code will be checked against a different dataset.
    keys = process_map(OSM_FILE)
    pprint.pprint(keys)
    #assert keys == {'lower': 5, 'lower_colon': 0, 'other': 1, 'problemchars': 1}


if __name__ == "__main__":
    test()

{'lower': 234007, 'lower_colon': 111321, 'other': 7884, 'problemchars': 0}


## Typecasting

In [39]:
def get_user(element):
    return


In [40]:

# typecasting
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if (element.tag == "node" or element.tag == "way" or element.tag == "relation" ):
            users.add(element.attrib['uid'])
            
            element.attrib['id'] =   int(float(element.attrib['id']))
            
            #element.attrib['lat'] =  float(element.attrib['lat'])
            #element.attrib['lon'] =  float(element.attrib['lon'])
            
            element.attrib['uid'] =   int(float(element.attrib['uid']))
            element.attrib['version'] = int(float(element.attrib['version']))
            element.attrib['changeset'] = int(float(element.attrib['changeset'])) 
            element.attrib['timestamp'] = str(element.attrib['timestamp'])
            
           

    return users



In [41]:
def test():

    users = process_map(SAMPLE_FILE)
    pprint.pprint(len(users))
   

In [42]:
if __name__ == "__main__":
    test()

440


# Problems Encountered in the Map

### Improving Street Names

In [61]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


In [62]:
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

In [63]:
# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.": "Road"
            }

In [70]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    
    if m:
        street_type = m.group()
        print(street_type)
        if street_type not in expected:
            street_types[street_type].add(street_name)

In [71]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


In [72]:
def audit(osmfile):
    osm_file = open(osmfile)
    street_types = collections.defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                                
                if is_street_name(tag):
                    
                    audit_street_type(street_types, tag.attrib['v'])
                    
                    
    osm_file.close()
    return street_types

In [73]:
def update_name(name, mapping):

    
    for key,value in mapping.iteritems():
        key_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
        m = key_type_re.search(name)
        if m:
            key_type = m.group()
        if key_type in mapping.keys():
            # substitute the street_type for its clean version in 'name'
            name = re.sub( key_type, mapping[key_type], name)
            

        return name


In [74]:
def test():
    st_types = audit(OSM_FILE)
    #assert len(st_types) == 3
    #pprint.pprint(dict(st_types))

    

if __name__ == '__main__':
    test()

Road
Southeast
102
Street
Boulevard
Boulevard
Boulevard
Street
Boulevard
Southeast
E
Street
Street
East
North
Place
Place
Place
Place
Place
Place
Place
Southwest
S
S
Street
S
Street
S
N
St
S
Street
Street
S
South
South
South
St
S
N
N
Way
Southwest
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
South
South
South
South
South
South
South
South
South
Street
Street
Drive
Drive
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Place
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
South
South
South
South
South
South
South
South
South
South
South
South
South
Street
Street
Street
Street
Street
Street
Street
Street
South

South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Stre

South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
South
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Stre

Avenue
Avenue
Avenue
Avenue
Avenue
Southwest
Meridian
Southwest
Southeast
East
Southeast
Drive
East
East
Way
East
East
East
East
East
Road
Avenue
Northeast
Way
Road
South
South
Southeast
South
South
Street
Street
Street
South
South
Street
Street
South
Southeast
Southeast
Southeast
Southeast
Northwest
Avenue
Avenue
Avenue
Avenue
East
East
East
South
Road
East
East
E
East
Southwest
Place
Way
Road
Parkway
West
1
Street
Southeast
Southeast
Street
Southeast
Southeast
Street
Southeast
Southeast
Street
Street
Place
Southeast
Place
Southeast
Street
Street
Southeast
Street
Street
Street
Southeast
Street
Street
Street
Street
Street
Street
Southeast
Street
Street
Street
Street
Southeast
Place
Street
Street
Street
Place
Street
Street
Street
Southeast
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Street
Road
Street
Street
Street
Place
Place
Street
Street
Street
Southeast
Southeast
Southeast
Southeast
South

## Improving Postal Codes

In [17]:
def is_postal_code(elem):
    return (elem.attrib['k'] == "addr:postcode")

In [23]:
def audit_postal_code(filename):
    osmfile = open(filename)
    
    for event, elem in ET.iterparse(osmfile, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postal_code(tag):
                    if len(tag.attrib['v']) > 5:
                        print(tag.attrib['v'])
    osmfile.close()
    
    

In [24]:
def test():
    audit_postal_code(SAMPLE_FILE)

if __name__ == '__main__':
    test()    

98057-4040


## Preparing for Database

In [34]:
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"
#import schemaf
import schema
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema
print(type(SCHEMA))


<type 'dict'>


In [35]:
# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

In [65]:
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
   

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    index = 0
    for child in element:
        if (child.tag == 'tag'):
            m = LOWER_COLON.search(child.attrib['k'])
            colon_type = []
            
            if m:
                colon_type = m.group()
            
            if (child.attrib['k'] in colon_type):
                colon_keys = child.attrib['k'].split(':')
                tags.append({'id': element.attrib['id'],
                'key': colon_keys[1],
                'value': child.attrib['v'],
                'type':colon_keys[0] })
            
            if (child.attrib['k'] not in colon_type ):
                if (child.attrib['k'].find(':')!=-1):
                    keys = child.attrib['k'].split(':',1)
                    tags.append({'id': element.attrib['id'],
                                'key': keys[1],
                                'value': child.attrib['v'],
                                 'type': keys[0] })
                else:
                    tags.append({'id': element.attrib['id'],
                                 'key': child.attrib['k'],
                                 'value': child.attrib['v'],
                                 'type': default_tag_type})
                    
        if (child.tag == 'nd'):
            way_nodes.append({'id':element.attrib['id'],
                              'node_id':child.attrib['ref'],
                              'position':index})
            index += 1;
           
    if element.tag == 'node':
        for field in node_attr_fields:
            node_attribs[field] = element.attrib[field]
        node_result = {}    
        for key,value in node_attribs.items():
            if value not in node_result.values() and value != " ":
                node_result[key] = value    
            
        return {'node': node_result, 'node_tags': tags}
    elif element.tag == 'way':
        for field in way_attr_fields:
            way_attribs[field] = element.attrib[field]
    
        way_result = {}
        
        for key,value in way_attribs.items():
            if value not in way_result.values():
                way_result[key] = value
            
        return {'way': way_result, 'way_nodes': way_nodes, 'way_tags': tags}


In [66]:

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    
    
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        
        if event == 'end' and elem.tag in tags:
            
            yield elem
            root.clear()



In [67]:

def validate_element(element, validator, schema=SCHEMA):
    #print(schema)
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


In [68]:
class UnicodeDictWriter(csv.DictWriter, object):
   

    def writerow(self, row):
        #print(row)
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
            
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [72]:
def process_map(file_in, validate):
      

    with codecs.open(NODES_PATH, 'wb') as nodes_file,\
    codecs.open(NODE_TAGS_PATH, 'wb') as nodes_tags_file,\
    codecs.open(WAYS_PATH, 'wb') as ways_file,\
    codecs.open(WAY_NODES_PATH, 'wb') as way_nodes_file,\
    codecs.open(WAY_TAGS_PATH, 'wb') as way_tags_file:
        
        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            #print(element)
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                    
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])




In [74]:

if __name__ == '__main__':
    
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    
    process_map(SAMPLE_FILE, validate=True)

In [7]:
import sqlite3

In [8]:
db = sqlite3.connect("C:\sqlite-windows\sqlite_windows\seattle.db")
c = db.cursor()

In [7]:
query = "select * from nodes where user = 'Skybunny'"
c.execute(query)
rows = c.fetchall()
print(rows)

# Problems in the map data

## Postal codes

In [29]:
query = "SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL \
         SELECT * FROM ways_tags) tags \
         WHERE tags.key='postcode'\
         GROUP BY tags.value\
         ORDER BY count DESC;"
c.execute(query)
rows = c.fetchall()

In [30]:
print(rows)

[(u'98178', 249), (u'98118', 114), (u'98057', 23), (u'98371', 19), (u'98092', 13), (u'98042', 11), (u'98032', 10), (u'98001', 4), (u'98030', 4), (u'98354', 4), (u'98010', 2), (u'98027', 2), (u'98038', 2), (u'98055', 2), (u'98058', 2), (u'98059', 2), (u'98372', 2), (u'98391', 2), (u'90092', 1), (u'98003', 1), (u'98032-1762', 1), (u'98057-4040', 1), (u'98104', 1), (u'98108', 1), (u'98168', 1), (u'98188', 1), (u'98198', 1), (u'98321', 1), (u'98373', 1), (u'98374', 1)]


## Sort Cities by count

In [53]:
query = "SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION  \
         SELECT * FROM ways_tags) tags\
         WHERE tags.key = 'city'\
         GROUP BY tags.value\
         ORDER BY count DESC;"
c.execute(query)
rows = c.fetchall()

In [54]:
print(rows)

[(u'Seattle', 365), (u'Renton', 29), (u'Kent', 25), (u'Puyallup', 24), (u'Auburn', 17), (u'Milton', 4), (u'Bonney Lake', 3), (u'Maple Valley', 3), (u'Black Diamond', 2), (u'Covington', 2), (u'Edgewood', 2), (u'Sumner', 2), (u'Algona', 1), (u'Buckley', 1), (u'Des Moines', 1), (u'Pacific', 1), (u'SeaTac', 1)]


# Overview of Data

File Sizes

Seattlemap.osm ......... 129 MB
sample.osm ............. 13 MB
seattle.db ............. 7 MB
nodes.csv .............. 4 MB
nodes_tags.csv ......... 267 KB
ways.csv ............... 437 KB
ways_tags.csv .......... 974 KB
ways_nodes.cv .......... 1.7 MB  

### Number of Nodes

In [11]:
query = "select count(*) from nodes"
c.execute(query)
rows = c.fetchall()
print(rows)

[(56861,)]


### Number of Ways

In [12]:
query = "select count(*) from ways"
c.execute(query)
rows = c.fetchall()
print(rows)

[(7266,)]


### Number of Unique Users

In [13]:
query = "select count(distinct(e.uid)) from(select uid from nodes UNION ALL select uid from ways) e"
c.execute(query)
rows = c.fetchall()
print(rows)

[(436,)]


### Top 10 Contributing Users

In [14]:
query = "select e.user,count(*) as num from(select user from nodes UNION ALL select user from ways) e 
        GROUP BY e.user
        ORDER BY num DESC
        LIMIT 10;"
c.execute(query)
rows = c.fetchall()
print(rows)

[(u'Omnific', 19989), (u'Grauer Elefant', 4687), (u'Amoebabadass', 4468), (u'STBrenden', 3977), (u'Glassman', 3680), (u'csytsma', 2836), (u'woodpeck_fixbot', 2403), (u'Geodesy99', 2245), (u'Skybunny', 1766), (u'zephyr', 1414)]


### Number of Users Appearing only once(having 1 post)

In [18]:
query = "SELECT COUNT(*) FROM (SELECT e.user, COUNT(*) as num\
        FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e\
        GROUP BY e.user HAVING num=1)  u;"
c.execute(query)
rows = c.fetchall()
print(rows)

[(107,)]


# Additional Ideas

### Top 10 Appearing Amenities

In [19]:
query = "SELECT value, COUNT(*) as num FROM nodes_tags\
        WHERE key='amenity'\
        GROUP BY value\
        ORDER BY num DESC\
        LIMIT 10;"
c.execute(query)
rows = c.fetchall()
print(rows)

[(u'restaurant', 42), (u'bench', 38), (u'fast_food', 34), (u'waste_basket', 20), (u'cafe', 19), (u'bank', 15), (u'doctors', 15), (u'fuel', 13), (u'dentist', 12), (u'school', 12)]


### Biggest Religion

In [20]:
query = "SELECT nodes_tags.value, COUNT(*) as num\
         FROM nodes_tags \
         JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') i\
         ON nodes_tags.id=i.id \
         WHERE nodes_tags.key='religion' \
         GROUP BY nodes_tags.value \
         ORDER BY num DESC \
         LIMIT 1;"
c.execute(query)
rows = c.fetchall()
print(rows)

[(u'christian', 6)]


### Most Popular Cuisines

In [21]:
query = "SELECT nodes_tags.value, COUNT(*) as num \
        FROM nodes_tags \
        JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i \
        ON nodes_tags.id=i.id \
        WHERE nodes_tags.key='cuisine' \
        GROUP BY nodes_tags.value \
        ORDER BY num DESC;"
c.execute(query)
rows = c.fetchall()
print(rows)

[(u'pizza', 7), (u'japanese', 4), (u'chinese', 3), (u'asian', 2), (u'burger', 2), (u'mexican', 2), (u'thai', 2), (u'american;savory_pancakes;pancake;breakfast', 1), (u'barbecue', 1), (u'burger;asian', 1), (u'indian', 1), (u'italian', 1), (u'pancake;breakfast', 1), (u'vietnamese', 1)]
